Python_Data_Science_2(Text Mining)

In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
import pandas as pd
import numpy as np
#from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/spambase.data', engine='python', header=None)

In [3]:
data = data.sample(frac = 1).reset_index(drop = True)
Xtrain = data.iloc[:-100,:-1]
Ytrain = data.iloc[:-100,-1]

Xtest = data.iloc[-100:,:-1]
Ytest = data.iloc[-100:,-1]

data = pd.read_csv('spambase.data').values # use pandas for convenience # gives a matrix
np.random.shuffle(data) #inplace shuffling
or
data = pd.read_csv('spambase.data').as_matrix() # use pandas for convenience # gives a matrix

In [4]:
# Naive Bayes
model = MultinomialNB()
model.fit(Xtrain, Ytrain)
print("Classification rate for NB:", model.score(Xtest, Ytest))

Classification rate for NB: 0.79


In [5]:
# AdaBoost
model2 = AdaBoostClassifier()
model.fit(Xtrain, Ytrain)
print("Classification rate for Adaboost:", model.score(Xtest, Ytest))

Classification rate for Adaboost: 0.79


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

In [7]:
spam_df = pd.read_csv('C:/Users/rpothams/Downloads/BD/NLP/spam.csv', encoding= 'iso-8859-1')
spam_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
spam_df = spam_df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
spam_df.columns = ['labels', 'data']

In [9]:
#creating binary labels
spam_df['b_labels'] = spam_df['labels'].map({'ham': 0, 'spam': 1})
Y = spam_df['b_labels'].values #numpy array

In [10]:
sentences = spam_df['data']
# using CountVectorizer
count_vect = CountVectorizer(decode_error='ignore')
X_count = count_vect.fit_transform(sentences)

In [11]:
tfidf = TfidfVectorizer()
# fit and then transform
tfidf.fit(sentences)
X_tfidf = tfidf.transform(sentences) #gives a numpy sparse matrix
#both fit and transform in one step
#X = tfidf.fit_transform(sentences)

In [12]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_tfidf, Y, test_size = 0.33)
Xtrain2, Xtest2, Ytrain2, Ytest2 = train_test_split(X_count, Y, test_size = 0.33)

In [13]:
print(Xtrain2[1])

  (0, 4218)	1
  (0, 7621)	1
  (0, 2689)	1
  (0, 8330)	1
  (0, 984)	1
  (0, 7659)	1
  (0, 2214)	1


In [14]:
model_countV = MultinomialNB()
model_countV.fit(Xtrain2, Ytrain2)
print("train score:", model_countV.score(Xtrain2, Ytrain))
print("test score:", model_countV.score(Xtest2, Ytest))

train score: 0.7645325475488883
test score: 0.7792278412180533


In [15]:
model_tfidf = MultinomialNB()
model_tfidf.fit(Xtrain, Ytrain)
print("train score:", model_tfidf.score(Xtrain, Ytrain))
print("test score:", model_tfidf.score(Xtest, Ytest))

train score: 0.9649075810340209
test score: 0.9575856443719413


In [ ]:
# wordcloud
from wordcloud import WordCloud
def visualize(df, label):
    words = ''
    for msg in df[df.iloc[:,1] == label].iloc[:,0]:
        msg = msg.lower()
        words += msg + ' '
    wordcloud = WordCloud(width=600, height=400).generate(words)
    plt.imshow(worcloud) #draws an image on the current figure (creating a figure is there isn't a current figure)
    plt.show()

In [ ]:
-*-**

#Section 3 - Psentiment Analysis
XML Parser - BeautifulSoup

#### Tokenization
why not just use .split for tokenization?
    contractions - combination of multiple words
        don't = do not
        won't = would not
        she'll = she will
    Extract punctuations and symbols: Period (.) and Dollar ($)
    This is all done by nltk.tokenize.word_tokenize

Custom Tokenizer (to perform better than in built nltk)
    - remove stopwords
    - remove numbers and single-letter tokens
    - application specific
    
#### Tokens into Vectors
    - scikitLear has it inbuilt
    - lets write it manually
    - each word is a key and index is a value - Dictionary/map
    

In [ ]:
# if the word is present, turn the value to 1, irresepective of frequency
V = 2000 #Vocab size
x = np.zeros(V)
for token in sentence:
    index = word_index_map[token]
    x[index] = 1
    #x[index] += 1 # for raw counts

In [23]:
# For sentiment analysis, we'll use raw counts, but qhat's the problem?
# Really long docs become really long vectors, really short docs have short vectors (ex: amazon long and short reviews)
# classifier have to deal with vectors of varying lengths and we dont want comparison based on length of vectors, hnce normalize
# Vector/# of words in document

# Another reason for normalization
# LogisticRegression performs better with normalizaed data (sigmoid(10) ~= sigmoid(100) ~= 1)

import nltk
from nltk.stem import WordNetLemmatizer # turns words in to base forms i.e dogs -> dog
from sklearn.linear_model import LogisticRegression

from bs4 import BeautifulSoup

wordnet_lemmatizer = WordNetLemmatizer()

stopwords = set(w.rstrip() for w in open('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/stopwords.txt'))

# positive.review is a XML file with Amazon positive reviews
positive_reviews = BeautifulSoup(open('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/electronics/positive.review'), 'xml').read()
positive_reviews = positive_reviews.findall('review_text')
negative_reviews = BeautifulSoup(open('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/electronics/negative.review')).read()
negative_reviews = negative_reviews.findall('review_text')

np.random.shuffle(positive_reviews)
#positive_reviews = positive_reviews[]

TypeError: 'NoneType' object is not callable

In [21]:
open('C:/Users/rpothams/Downloads/BD/NLP/nlp_class/electronics/positive.review')

<_io.TextIOWrapper name='C:/Users/rpothams/Downloads/BD/NLP/nlp_class/electronics/positive.review' mode='r' encoding='cp1252'>